In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder \
    .appName("myapp") \
    .master("local[*]")\
    .getOrCreate()

In [3]:
sc=spark.sparkContext

In [4]:
print("spark version:",spark.version)

spark version: 3.5.5


In [5]:
df=spark.read.csv('final_data (2).csv',header=True,inferSchema=True,mode="DROPMALFORMED")
df.show()

+---------------------+-------------------+--------------------+--------------+------+-----------+---------+------+--------------------+-----------------+-----+-----+------------------+------------------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|trans_date_trans_time|             cc_num|            merchant|      category|   amt|      first|     last|gender|              street|             city|state|  zip|               lat|              long|city_pop|                 job|       dob|           trans_num| unix_time|         merch_lat|        merch_long|is_fraud|
+---------------------+-------------------+--------------------+--------------+------+-----------+---------+------+--------------------+-----------------+-----+-----+------------------+------------------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|  2019-08-11 19:38:33|  

In [8]:
df.distinct().count()

19302

In [9]:
df.printSchema()

root
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)



In [10]:
df.select("city").show(5)

+------------+
|        city|
+------------+
| Moundsville|
|Smiths Grove|
|     Freedom|
|    Grantham|
|       Paris|
+------------+
only showing top 5 rows



In [11]:
fraud_df = df.filter(df.is_fraud == 1)
fraud_df.show()


+---------------------+----------------+--------------------+-------------+-------+-------+------+------+--------------------+-------------+-----+-----+-------+---------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|trans_date_trans_time|          cc_num|            merchant|     category|    amt|  first|  last|gender|              street|         city|state|  zip|    lat|     long|city_pop|                 job|       dob|           trans_num| unix_time|         merch_lat|        merch_long|is_fraud|
+---------------------+----------------+--------------------+-------------+-------+-------+------+------+--------------------+-------------+-----+-----+-------+---------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|  2019-01-02 01:06:37|   4613314721966|fraud_Rutherford-...|  grocery_pos| 281.06|  Jason|Murphy|     M|542 Steve Curve S...|C

what are the 10 most frequent merchants for fraudulant transactions

In [12]:
merchant_fraud_counts = fraud_df.groupBy("Merchant").count().orderBy("count", ascending=False).show(10)

+--------------------+-----+
|            Merchant|count|
+--------------------+-----+
|   fraud_Kilback LLC|   62|
|  fraud_Rau and Sons|   60|
|   fraud_Kozey-Boehm|   60|
|     fraud_Doyle Ltd|   57|
|    fraud_Terry-Huel|   56|
|      fraud_Kuhn LLC|   55|
|     fraud_Boyer PLC|   55|
|     fraud_Kuhic LLC|   53|
|fraud_Moen, Reing...|   53|
|fraud_Kiehn-Emmerich|   53|
+--------------------+-----+
only showing top 10 rows



In [16]:
state_transaction_counts = df.groupBy("state").count().orderBy("count", ascending=False).show(10)

+-----+-----+
|state|count|
+-----+-----+
|   NY| 1290|
|   TX| 1287|
|   PA| 1153|
|   CA|  857|
|   OH|  687|
|   FL|  656|
|   IL|  651|
|   MI|  632|
|   AL|  578|
|   MO|  534|
+-----+-----+
only showing top 10 rows



In [14]:
state_transaction_counts = df.groupBy("state").agg({"trans_num": "count"}).withColumnRenamed("count(trans_num)", "Transaction_Count").orderBy(col("Transaction_Count"), ascending=False).show(5)

NameError: name 'col' is not defined

In [25]:
from pyspark.sql.functions import col, when, avg, year, months_between

In [42]:
df_age = df.withColumn("age", year(col("trans_date_trans_time")) - year(col("dob")))
df_age = df_age.withColumn("age_group",
    when(col("age").between(18, 25), "18-25")
    .when(col("age").between(26, 35), "26-35")
    .when(col("age").between(36, 45), "36-45")
    .when(col("age").between(46, 55), "46-55")
    .when(col("age").between(56, 65), "56-65")
    .otherwise("66+")
)
age_group_count = df_age.groupBy("age_group").count()
age_group_count.show()
age_spending_df = df_age.groupBy("age_group").agg(avg("amt").alias("avg_spending"))
age_spending_df.show()

+---------+-----+
|age_group|count|
+---------+-----+
|    18-25| 1759|
|    26-35| 4057|
|    46-55| 3822|
|    36-45| 3490|
|    56-65| 2842|
|      66+| 3332|
+---------+-----+

+---------+------------------+
|age_group|      avg_spending|
+---------+------------------+
|    18-25| 335.2217907902219|
|    26-35| 266.2523638156277|
|    46-55|  296.649068550497|
|    36-45|248.35661604584544|
|    56-65| 338.6428888106971|
|      66+|344.00427070828346|
+---------+------------------+



In [17]:
from pyspark.sql.functions  import year,current_date

In [39]:
from pyspark.sql.functions import col, when, avg

In [41]:
df_age_find=df.select("dob","first","last","amt", (year(current_date())-year(col("dob"))).alias("age_transaction")).orderBy("first")
df_age_find.show()

+----------+-----+-------+------+---------------+
|       dob|first|   last|   amt|age_transaction|
+----------+-----+-------+------+---------------+
|1950-11-27|Aaron|   Pena| 285.8|             75|
|1950-11-27|Aaron|   Pena|  7.64|             75|
|1950-11-27|Aaron|   Pena|960.14|             75|
|1974-12-23|Aaron| Murray|719.82|             51|
|1950-11-27|Aaron|   Pena|1060.9|             75|
|1945-03-15|Aaron| Rogers| 23.81|             80|
|1950-11-27|Aaron|   Pena|890.94|             75|
|1974-12-23|Aaron| Murray| 49.98|             51|
|1950-11-27|Aaron|   Pena| 915.2|             75|
|1974-12-23|Aaron| Murray| 50.14|             51|
|1950-11-27|Aaron|   Pena|773.33|             75|
|1945-03-15|Aaron| Rogers| 61.35|             80|
|1950-11-27|Aaron|   Pena|751.72|             75|
|1945-03-15|Aaron| Rogers| 64.56|             80|
|1950-11-27|Aaron|   Pena|  1.31|             75|
|1995-04-22|Aaron|Stewart| 90.27|             30|
|1945-03-15|Aaron| Rogers| 85.05|             80|


In [42]:
df_age = df_age_find.withColumn("age_group",
    when(col("age_transaction").between(18, 25), "18-25")
    .when(col("age_transaction").between(26, 35), "26-35")
    .when(col("age_transaction").between(36, 45), "36-45")
    .when(col("age_transaction").between(46, 55), "46-55")
    .when(col("age_transaction").between(56, 65), "56-65")
    .otherwise("66+")
)


In [43]:
age_group_count = df_age.groupBy("age_group").count()
age_group_count.show()


+---------+-----+
|age_group|count|
+---------+-----+
|    18-25|  405|
|    26-35| 3084|
|    46-55| 3986|
|    36-45| 3985|
|    56-65| 3222|
|      66+| 4620|
+---------+-----+



In [44]:

age_spending_df = df_age.groupBy("age_group").agg(avg("amt").alias("avg_spending"))
age_spending_df.show()

+---------+------------------+
|age_group|      avg_spending|
+---------+------------------+
|    18-25|341.13350617283965|
|    26-35| 300.2741893644618|
|    46-55|244.01207727044675|
|    36-45| 256.7721430363862|
|    56-65| 354.1269646182494|
|      66+|341.55254112554127|
+---------+------------------+

